In [ ]:
# Import libraries
!pip install pandas-profiling
!pip install ydata-profiling
!pip install geopy
!pip install autoviz
!pip install matplotlib
!pip install -U textblob
!pip install statsmodels
!pip install missingno
!python -m textblob.download_corpora
!pip install missingpy
!pip install scikit-learn
!pip install sklearn
import pickle
import re
import numpy as np 
import seaborn as sns
import plotly.graph_objects as go
import pandas as pd
import missingno as msno
import matplotlib.pyplot as plt
import os
import sklearn
import joblib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is alread

In [ ]:
# Set path to the directory containing the training dataset
path_copy_test = "" # add specific path
df_test_or = pd.read_csv(path_copy_test) # Getting the original training data Set
df_test = df_test_or.copy() # Keeping an exact copy from the data set

In [ ]:
df = df_test

#### Defining functions to use

In this section, the used functions are defined. Make sure to run it before the rest of the code. 

EDA

In [ ]:
# Defining numerical and categorical features of the data set 
numerical_features = [
'property_max_guests', 'host_response_rate', 'booking_max_nights', 
'booking_min_nights', 'booking_availability_30', 
'booking_availability_60', 'booking_availability_90',
'booking_availability_365', 'reviews_num', 'reviews_rating', 'reviews_acc', 'reviews_cleanliness',
'reviews_checkin', 'reviews_communication', 'reviews_location',
'reviews_value', 'reviews_per_month', 'target'
]
categorical_features = [
'property_summary', 'property_space',
'property_desc', 'property_neighborhood', 'property_notes',
'property_zipcode', 'property_lat', 'property_lon',
'property_type', 'property_room_type',
'property_bathrooms', 'property_bedrooms', 'property_beds',
'property_bed_type', 'property_amenities', 'property_sqfeet',
'property_last_updated', 'host_id',
'host_location', 'host_about', 'host_response_time',
'host_nr_listings', 'host_nr_listings_total',
'host_verified', 'booking_price_covers', 'booking_cancel_policy',
'extra', 'property_id', 'property_name', 'property_transit', 
'property_access', 'property_interaction', 'property_rules', 
'property_scraped_at', 'host_since', 'reviews_first', 'reviews_last'
]

text_columns = [
    'property_name', 'property_summary', 'property_space',
                'property_desc', 'property_neighborhood', 'property_notes',
                'property_transit', 'property_access', 'property_interaction',
                'property_rules'
] # The text columns will be treated separately for featuring later on

In [ ]:
# Function for obtaining the descriptive statistics about the target feature
def stats_target(df, target):
  target_stats = df[target].describe() # Getting the target's descriptive statistics
  stats_target_df = pd.DataFrame(target_stats) # Getting a dataframe
  return stats_target_df

In [ ]:
# Function for obtaining the best distribution of target variable
from scipy import stats

def find_best_distribution(data, feature):
    # Fit different distributions to the data
    distributions = [
        stats.norm,  # Normal distribution
        stats.expon,  # Exponential distribution
        stats.gamma,  # Gamma distribution
        stats.lognorm,  # Lognormal distribution
        stats.beta,  # Beta distribution
        stats.uniform  # Uniform distribution
    ]

    # Compute the AIC for each distribution
    aic_values = []
    for distribution in distributions:
        params = distribution.fit(data[feature])
        log_likelihood = distribution.logpdf(data[feature], *params).sum()
        aic = -2 * log_likelihood + 2 * len(params)
        aic_values.append(aic)

    # Select the distribution with the lowest AIC
    best_distribution = distributions[np.argmin(aic_values)]

    # Return the name of the best fitting distribution
    return best_distribution.name

In [ ]:
import seaborn as sns

def univariate_plotting(df, feature):
    # Plot histogram with KDE
    plt.figure(figsize=(8, 6))
    sns.histplot(data=df, x=feature, kde=True, bins=30)
    plt.xlabel('Measurement')
    plt.ylabel('Frequency')
    plt.title('Histogram')
    plt.show()

    # Plot box plot
    plt.figure(figsize=(8, 6))
    sns.boxplot(data=df, x=feature, notch=True, sym='o')
    plt.xlabel('Group')
    plt.ylabel('Measurement')
    plt.title('Box Plot')
    plt.show()

    # Plot kernel density estimate
    plt.figure(figsize=(8, 6))
    sns.kdeplot(data=df, x=feature)
    plt.xlabel('Measurement')
    plt.ylabel('Density')
    plt.title('Kernel Density Plot')
    plt.show()

In [ ]:
# Definying a function for correlation identification with heatmap visualization
def correlation_heatmap(df):
    # Getting correlation matrix
    corr_matrix = df.corr()
    # Print correlation matrix
    print("Correlation Matrix:\n", corr_matrix.to_string())
    # Create heatmap plot
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr_matrix, cmap='coolwarm', annot=True, vmin=-1, vmax=1, fmt='.1f')
    # Show the plot
    plt.show()

In [ ]:
# Defining a function for obtaining scatterplots from all numerical features
def scatterplots(df, numerical_features, target_feature):
    for feature in numerical_features:
        if feature != target_feature:
            plt.figure()
            plt.scatter(df[feature], df[target_feature])
            plt.xlabel(feature)
            plt.ylabel(target_feature)
            plt.title(f"Scatterplot: {feature} vs {target_feature}")
            plt.show()

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage
# Defining a function for doing clustering on two categorical features
def plot_dendrogram(dataframe):
    # Compute the linkage matrix using Ward's method
    linkage_matrix = linkage(dataframe, method='ward')

    # Plot the dendrogram
    fig, ax = plt.subplots(figsize=(10, 5))
    dendrogram(linkage_matrix)
    plt.xlabel('Locations')
    plt.ylabel('Distance')
    plt.show()

In [ ]:
# Getting the accurate types for the features within the data set 
def convert_correct_types(df):
    if df['property_bedrooms'].dtype == 'float64':
        df['property_bedrooms'] = df['property_bedrooms'].fillna(np.NaN).astype('Int64')
    if df['property_beds'].dtype == 'float64':
        df['property_beds'] = df['property_beds'].fillna(np.NaN).astype('Int64')
    if df['booking_price_covers'].dtype == 'float64':
        df['booking_price_covers'] = df['booking_price_covers'].fillna(np.NaN).astype('Int64')
    if df['host_response_rate'].dtype == 'float64':
        df['host_response_rate'] = df['host_response_rate'].fillna(np.NaN).astype('Int64')
    return df 

In [ ]:
# Defining function for identifying identical features
def duplicate_features(data):
    duplicates = set()  
    kept_features = []
    removed_features = []  

    for i, column1 in enumerate(data.columns):  
        for j, column2 in enumerate(data.columns[i + 1:], i + 1):
            # Checking if column1 and column2 are equal
            if data[column1].equals(data[column2]):
                # If they are equal, add column2 to the duplicates set
                duplicates.add(column2)
                # Get the kept feature 
                kept_feature = data.columns[i]
                kept_features.append(kept_feature)  # List of the kept features
                removed_features.append(column2)

                # Print the duplicated feature
                print(f"Feature '{column2}' is duplicated.")

Missigness

In [ ]:
# Defining function for recognizing missing values within features
def features_missing_values(df):
    missing = df.isnull().sum() # Identifying missing sum within each feature
    missing_pct = (missing / len(df)) * 100 # Percentage of missingness compare to total n in dataset
    missing_values = pd.concat([
        missing, missing_pct.astype('int64')], # Getting type int64 for Percentage column 
        axis=1, 
        keys=[
            'Total Missing', 'Percentage (%)']
     ) # Creating dataframe with % of missing for each feature
    missing_values.sort_values(by='Total Missing', ascending=False, inplace=True)
    missing_values.index.name = 'Feature'
    return missing_values 

In [ ]:
# Defining function for identifying the features with missing rates greater than 90%
def identify_missing_features(data, threshold=0.9):
    missing_perc = data.isnull().sum() / len(data) * 100  # Calculate the percentage of missingness within each feature
    high_missing_features = missing_perc[missing_perc >= threshold * 100].index.tolist()  # Features with missing rate >= 90%

    if not high_missing_features:
        print("No features with missingness rate greater than {}% found. The default threshold is 90%.".format(threshold * 100))
    else:
        print("The following features have a missing rate greater than {}%:".format(threshold * 100))
        for feature in high_missing_features:
            print("- {}".format(feature))

In [ ]:
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
# Defining a function for missing imputation using KNN and SimpleImputer (mode)
def impute_missing_values(df):
    imputed_df = df.copy()

    numerical_features_remaining = [col for col in numerical_features if col in imputed_df.columns]
    categorical_features_remaining = [col for col in categorical_features if col in imputed_df.columns]

    knn_imputer = KNNImputer(n_neighbors=5)

    imputed_numerical = knn_imputer.fit_transform(imputed_df[numerical_features_remaining])
    imputed_numerical_df = pd.DataFrame(imputed_numerical, columns=numerical_features_remaining)
    imputed_numerical_df[numerical_features_remaining] = imputed_numerical_df[numerical_features_remaining].astype(imputed_df[numerical_features_remaining].dtypes)

    imputed_df[numerical_features_remaining] = imputed_numerical_df[numerical_features_remaining]

    categorical_features_to_impute = [col for col in categorical_features_remaining if col not in text_columns]
    mode_imputers = {}
    for col in categorical_features_to_impute:
        mode_imputer = SimpleImputer(strategy='most_frequent')
        imputed_df[col] = mode_imputer.fit_transform(imputed_df[col].values.reshape(-1, 1))
        mode_imputers[col] = mode_imputer

    imputed = (df[numerical_features_remaining] != imputed_numerical_df[numerical_features_remaining]).any(axis=1)
    imputed_df['imputed'] = imputed.astype('int64')

    return imputed_df

In [ ]:
# Defining a function for the missing features with more than 5% of missingness
def features_missing_values(df, threshold=5):
    missing = df.isnull().sum() # Identifying missing sum within each feature
    missing_pct = (missing / len(df)) * 100 # Percentage of missingness compare to total n in dataset
    missing_values = pd.concat([
        missing, missing_pct.astype('int64')], # Getting type int64 for Percentage column 
        axis=1, 
        keys=[
            'Total Missing', 'Percentage (%)']
     ) # Creating dataframe with % of missing for each feature
    missing_values.sort_values(by='Total Missing', ascending=False, inplace=True) # Sorting
    missing_values.index.name = 'Feature' # Setting the index as 'Feature'
    return missing_values[missing_values['Percentage (%)'] > threshold].index.tolist()

outliers

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import LocalOutlierFactor
# Defining function doing one-hot encoding on categorical features
def encode_categorical_features(df, categorical_features):
    # Perform one-hot encoding on the categorical features
    encoder = OneHotEncoder(sparse=True, handle_unknown='ignore')
    onehot = encoder.fit_transform(df[categorical_features])
    onehot_labels = encoder.get_feature_names_out(categorical_features)

    # Creating a new DataFrame with the encoded features
    df_onehot = pd.DataFrame.sparse.from_spmatrix(onehot, columns=onehot_labels)

    return df_onehot
  
# Defining function for identifying outliers
def perform_outlier_detection(df, numerical_features, categorical_features, text_columns):
    # Filtering the numerical and categorical features
    numerical_features_remaining = [col for col in numerical_features if col in df.columns]
    categorical_features_remaining = [col for col in categorical_features if col in df.columns]
    features = numerical_features_remaining + list(df.columns[df.columns.str.startswith('x')]) # This excludes the text columns

    # Encoding categorical features (excluding text columns)
    encoded_df = encode_categorical_features(df[categorical_features_remaining], categorical_features_remaining)

    # Selecting the features
    features = [col for col in features if col in encoded_df.columns]

    # Performing outlier detection using Local Outlier Factor (LOF)
    lof = LocalOutlierFactor(n_neighbors=20, contamination=0.02)
    y_pred = lof.fit_predict(pd.concat([encoded_df[features], df[numerical_features_remaining]], axis=1))

    # Creating a new column to store outlier
    df['is_outlier'] = y_pred

    return df

In [ ]:
def visualize_outlier_detection(df):
    # Plotting the numerical features against each other
    sns.pairplot(df, vars=numerical_features, hue='is_outlier', plot_kws={'alpha': 0.6})

    # Adding legend
    plt.legend(title='Outlier', labels=['Normal', 'Outlier'])

    # Adjusting the layout
    plt.tight_layout()

    # Display the plot
    plt.show()

featuring

In [ ]:
# Create dummy column for every 'extra' (1 or 0 if property has this extra or not)
def add_and_populate_columns_for_extras(df):
  #step 1: sum up all possible amenities
  extras = [] #empty list
  for index,row in df.iterrows(): #loop over rows
    row_as_list = str(row['extra']).split(sep=', ') #split string seperate amenities (returns list)
    for extra in row_as_list: #loop over seperate amenities in list
      if not extra in extras and extra != 'nan': #for every (new) amenity, check whether it is already in the list, add if not
        extras.append(extra)

  #step 2: create and populate columns in df
  print(extras)
  for extra in extras:
    df['dummy_extra_' + extra] = df.apply(lambda x: extra in str(x['extra']).split(sep=', '), axis=1).astype(int) #loop over unique amenities and check whether string contains this amenity, if so, returns True (dummy=1)

In [ ]:
from geopy.geocoders import Nominatim
# Defining function for getting the zipcode from latitude and longitude on the missing values for those features 
# Getting a geolocator using Nominatim
geolocator = Nominatim(user_agent="zipcode2", timeout=10)

def add_zipcode2_column(df):
    if "zipcode2" in df.columns:
        print("Column 'zipcode2' already exists. Skipping computation.")
        return df
    else: # If the code has ran already and feature was created, no procedure is done
    # If feature has been created, then getting the zipcode from the coordinates on the data set. Make sure to define the same name on the features :)
        def get_zipcode2(row):
            if pd.notnull(row['property_zipcode']): # If the row has already the zipcode, keep it 
                return row['property_zipcode'].replace(" ", "") #delete spaces
            else: # If row does not have a zipcode, then obtaining it from the coordinates
                lat = row['property_lat']
                lon = row['property_lon']
                if pd.notnull(lat) and pd.notnull(lon):
                    location = geolocator.reverse(f"{lat},{lon}", exactly_one=True)
                    address = location.raw['address']
                    return address.get('postcode')
                else:
                    return None
                    t
        df['zipcode2'] = df.apply(get_zipcode2, axis=1)
        print("Column 'zipcode2' added to DataFrame.")
        return df

In [ ]:
# Defining a function for getting the city from the zipcode obtained
def city(data):
    for index, row in data.iterrows():
        zipcode = str(row['zipcode2'])
        if zipcode.startswith("1"):
            data.loc[index, 'city'] = "Brussels"
        elif zipcode.startswith("2"):
            data.loc[index, 'city'] = "Antwerp"
        else:
            data.loc[index, 'city'] = "Unknown"
    
    return data

In [ ]:
# Defining a function for grouping information about how the room is configurated
def group_rooms(df):
    grouped_df = df.copy()

    # Create a new feature for room configuration
    grouped_df['room_configuration'] = ''

    # Iterate over each row
    for index, row in grouped_df.iterrows():
        num_bathrooms = row['property_bathrooms']
        num_bedrooms = row['property_bedrooms']
        num_beds = row['property_beds']

        # Determine the room configuration based on the number of rooms
        if num_bathrooms == 1 and num_bedrooms == 1 and num_beds == 1:
            grouped_df.loc[index, 'room_configuration'] = '1B1B1B'  # 1 Bathroom, 1 Bedroom, 1 Bed
        elif num_bathrooms >= 2 and num_bedrooms >= 2 and num_beds >= 2:
            grouped_df.loc[index, 'room_configuration'] = '2B2B2B+'  # 2 or more Bathrooms, 2 or more Bedrooms, 2 or more Beds
        else:
            grouped_df.loc[index, 'room_configuration'] = 'Other'  # Other room configurations

    return grouped_df

In [ ]:
# Defining a function for calculating the rate among the max number of guests and property rooms 
def calculate_guests_to_rooms_ratio(df):
    # Calculate the sum of bathrooms, bedrooms, and beds
    df['rooms_sum'] = df['property_bathrooms'] + df['property_bedrooms'] + df['property_beds']

    # Calculate the guests to rooms ratio
    df['guests_to_rooms_ratio'] = df['property_max_guests'] / df['rooms_sum']

    return df

In [ ]:
# Defining a function for crating boolean columns as desired
def create_combined_boolean_column(df, columns, new_column_name):
    df[new_column_name] = (df[columns] == 1).any(axis=1).astype(int)
    return df

In [ ]:
# Defining a function for grouping the amenities into categories
def group_property_amenities(df):
    # Define categories and keywords/patterns
    categories = {
        'Kitchen': ['kitchen'],
        'Entertainment': ['cable', 'tv'],
        'Safety': ['smoke detector', 'carbon monoxide detector', 'first aid kit', 'fire extinguisher'],
        'Internet': ['internet', 'wireless internet'],
        'Pets': ['pet', 'dog', 'cat'],
        'Other amenities': []
    }

    # Group amenities into categories
    grouped_amenities = {category: [] for category in categories.keys()}
    default_category = 'Other amenities'

    for amenities in df['property_amenities']:
        if isinstance(amenities, str):
            for amenity in amenities.split(','):
                amenity = amenity.strip()
                for category, keywords in categories.items():
                    if any(re.search(keyword, amenity, re.IGNORECASE) for keyword in keywords):
                        grouped_amenities[category].append(amenity)
                        break
                else:
                    grouped_amenities[default_category].append(amenity)

    # Create binary features for each category
    for category, amenities in grouped_amenities.items():
        df[category] = df['property_amenities'].apply(lambda x: int(any(amenity in x for amenity in amenities)))

    return df

In [ ]:
from datetime import datetime
# Defining a function for calculating the duration of a host
def calculate_host_duration(df, host_since_column='host_since'):

    df[host_since_column] = pd.to_datetime(df[host_since_column])

    # Calculate the duration
    df['host_duration'] = (datetime.now() - df[host_since_column]).dt.days

    return df

In [ ]:
from geopy.distance import geodesic
# Defining function for calulating the distance of the property to the city center
def add_distance_to_city_center(df, city_center_coords, city_column='city'):
    for city, coords in city_center_coords.items():
        city_mask = df[city_column] == city
        df.loc[city_mask, 'distance_to_city_center'] = df[city_mask].apply(
            lambda row: geodesic((row['property_lat'], row['property_lon']), coords).kilometers,
            axis=1
        )

# Usage example
city_center_coords = {
    'Brussels': (50.8503, 4.3517),
    'Antwerp': (51.2194, 4.4025)
}

In [ ]:
# Getting a booking rate when comparing the max number of nights and the min nights against the booking availability
def calculate_booking_rate(df):
    df['booking_rate'] = df['booking_max_nights'] - df['booking_min_nights'] / df['booking_availability_365']
    df['booking_rate'] = df['booking_rate'].replace([np.inf, -np.inf], 0)
    return df

model evaluation

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# Defining function for getting the metrics for doing model selection
def score_estimator(estimator, X_test, y_test):
    """Score an estimator on the test set."""
    y_pred = np.round(estimator.predict(X_test))

    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    # Print each of the metrics
    print("MSE: %.3f" % mse)
    print("RMSE: %.3f" % rmse)
    print("MAE: %.3f" % mae)

# Preprocessing


## Data Cleaning and data consistency


In [ ]:
# Making values from zipcode uniform
if df['property_zipcode'].dtype == 'object':
    df['property_zipcode'] = df['property_zipcode'].str.strip()
    df['property_zipcode'] = df['property_zipcode'].str.replace(' ', '')

In [ ]:
# Identifying duplicated features
duplicate_features(df)

Feature 'host_nr_listings_total' is duplicated.


In [ ]:
# Dropping the identified duplicated feature
df = df.drop('host_nr_listings_total', axis=1)

## Dealing with missigness

In [ ]:
# Identifying features with missingness greater than 90%
identify_missing_features(df, threshold=0.9)

The following features have a missing rate greater than 90.0%:
- property_sqfeet


In [ ]:
# Dropping those features
df = df.drop('property_sqfeet', axis=1)

In [ ]:
# Performing KNN and simple imputation on the missing values
df_imputed = impute_missing_values(df)

## Dealing with outliers

In [ ]:
# Perform one-hot encoding and outlier detection
df_out = perform_outlier_detection(df_imputed, numerical_features, categorical_features, text_columns)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
# count the unique values of a feature
unique_counts = df_out['is_outlier'].value_counts()

# print the unique value counts
print(unique_counts)

 1    900
-1     19
Name: is_outlier, dtype: int64


In [ ]:
# Getting the rows considered to be outliers
removed_rows = df_out[df_out['is_outlier'] == -1].index.tolist()
df_out = df_out[df_out['is_outlier'] != -1]
# Removing the outliers from the data set
df_imputed = df_imputed.drop(removed_rows)
# obtaining a final data set
df = df_imputed
print("Removed rows:")
print(removed_rows)

Removed rows:
[7, 58, 82, 121, 136, 182, 411, 471, 476, 480, 498, 501, 722, 771, 773, 778, 782, 791, 917]


Uncoment if you want to perform it on obs <200 for the target

In [ ]:
# df = df[df['target'] <= 200]

## Creating features

In [ ]:
# Condition to check for text by row
has_text = df[text_columns].apply(lambda row: 0 if any(pd.isna(row)) or any(row == ' ') or any(row == ' ') else 1, axis=1)

# Assign value to text_var based on the condition
df['text_var'] = has_text

In [ ]:
# eEcode property_room_type into entire home/apt vs other
def binary_property_room_type(row):
    if row['property_room_type'] == 'Entire home/apt': 
        return 1
    else:
        return 0 
df['property_room_type'] = df.apply(lambda x: binary_property_room_type(x), axis=1)

print(df.property_room_type)

0      1
1      1
2      1
3      1
4      0
      ..
913    1
914    1
915    1
916    1
918    1
Name: property_room_type, Length: 900, dtype: int64


In [ ]:
# Setting an index for reviews: 'reviews_acc', 'reviews_cleanliness', 'reviews_checkin', 'reviews_communication', 'reviews_location', 'reviews_value'
df['reviews_avg'] = df[['reviews_acc', 'reviews_cleanliness', 'reviews_checkin', 'reviews_communication', 'reviews_location', 'reviews_value']].mean(axis=1)
print(df.reviews_avg)

0       9.166667
1       9.166667
2       8.500000
3       9.000000
4       9.266667
         ...    
913     8.666667
914    10.000000
915     9.833333
916     9.400000
918    10.000000
Name: reviews_avg, Length: 900, dtype: float64


In [ ]:
# Adding zipcodes from the latitude and longitude
df['property_zipcode'] = df['property_zipcode'].astype(str)
df = add_zipcode2_column(df)

Column 'zipcode2' added to DataFrame.


In [ ]:
# Getting the cities were property was located
df = city(df)

In [ ]:
df['city'].value_counts()

Brussels    785
Antwerp     115
Name: city, dtype: int64

In [ ]:
# Dummification on extra variable
add_and_populate_columns_for_extras(df)

['Host Has Profile Pic', 'Host Identity Verified', 'Is Location Exact', 'Instant Bookable', 'Require Guest Profile Picture', 'Require Guest Phone Verification', 'Host Is Superhost']


In [ ]:
# Calculating the distance of the property to the city center
add_distance_to_city_center(df, city_center_coords)

In [ ]:
# Getting only relevant extra characteristics
columns_to_check = ['dummy_extra_Is Location Exact', 'dummy_extra_Instant Bookable', 'dummy_extra_Host Identity Verified', 'dummy_extra_Host Is Superhost']
new_column_name = 'ext'

df = create_combined_boolean_column(df, columns_to_check, new_column_name)

In [ ]:
# Getting the room configuration
df = group_rooms(df)

In [ ]:
# Calculating the host duration
df = calculate_host_duration(df)

In [ ]:
# Getting the guesst rooms ratio
df = calculate_guests_to_rooms_ratio(df)

In [ ]:
# Getting the amenities categories
df = group_property_amenities(df)

In [ ]:
# Calculating the booking rate
df = calculate_booking_rate(df)

# Set up

In [ ]:
set_up = df[[
    'reviews_avg', 'booking_rate', 'reviews_per_month', 'guests_to_rooms_ratio', 'host_response_rate', 'host_duration', 'distance_to_city_center',
    'room_configuration', 'property_room_type', 'Entertainment',
]] # Same set-up, but target is excluded

In [ ]:
# Load the model from the file
filename = ' ' # add the path 
with open(filename, 'rb') as file:
    loaded_model = pickle.load(file)

In [ ]:
# Make predictions on the test data
y_pred = loaded_model.predict(df)

[66.14256693 83.79383944 65.48870563 68.04017811 62.51240972 62.51240972
 81.73947959 68.4584167  60.89482969 67.20564238 68.37995526 64.86620028
 57.80219613 66.82167784 75.18022453 66.43118861 66.98284866 66.95297993
 65.48441104 65.84102977 61.08446676 70.39735245 61.04167164 61.77832617
 71.58669051 66.26396679 61.81081213 68.23831956 65.48441104 64.60801276
 61.39353095 64.34634095 66.8108617  68.76013093 61.92687917 63.84146913
 61.53390646 66.4991083  68.51783351 68.35902716 65.48441104 65.12405683
 78.27910652 67.25864381 68.51595837 72.06845871 69.48943114 70.65662489
 70.17588288 75.18022453 73.87196871 71.00468477 69.32425057 68.25541486
 63.84146913 66.83657178 67.07920563 66.37267155 67.57413332 64.72667666
 60.92873532 68.47947395 68.23831956 67.35733349 65.74770351 68.31548599
 69.9268019  70.12219283 68.06548907 68.09006748 69.31693157 71.93906051
 75.09649108 71.93906051 68.06548907 68.09006748 68.06548907 69.73488222
 71.33266856 71.3482755  68.06548907 71.85394057 67